In [1]:
import boto3
import geojson
import pyarrow as pa
import pandas as pd
import geopandas as gpd

from typing import ClassVar, Type
from datetime import datetime
from pyarrow.fs import S3FileSystem
from dataclasses import dataclass

from pyiceberg.catalog import load_catalog
from pyiceberg.catalog.glue import GlueCatalog
from pyiceberg.table import StaticTable, DataScan
from pyiceberg.partitioning import PartitionSpec, PartitionField
from pyiceberg.table.sorting import SortOrder, SortField
from pyiceberg.io.pyarrow import pyarrow_to_schema
from pyiceberg.transforms import IdentityTransform, DayTransform, YearTransform
from pyiceberg.expressions import EqualTo, GreaterThanOrEqual
from pyiceberg.schema import Schema
from pyiceberg.table import Table
from pyiceberg import types as T

from geojson.utils import coords
from shapely.geometry import shape, Polygon, LinearRing, box, mapping

In [2]:
glue_catalog = load_catalog(
    'default',
    type='glue',
    profile='default',
    region_name='us-east-1',
    client={
        'region': 'us-east-1',
    }
)

In [3]:
# glue_catalog.create_namespace_if_not_exists("db_ice")

In [4]:
glue_catalog.list_namespaces()

[('base',), ('db_ice',), ('default',), ('sagemaker_featurestore',)]

In [5]:
DATABASE = "db_ice"

In [6]:
glue_catalog.list_tables(DATABASE)

[('db_ice', 'tb_controle_imagem'),
 ('db_ice', 'tb_controle_ingestao'),
 ('db_ice', 'tb_imagem'),
 ('db_ice', 'tb_poligono')]

## Create table

In [7]:
tb_name        = "tb_controle_ingestao"
table_location = f"s3://data-us-east-1-891377318910/db_ice/{tb_name}" 

schema = Schema(
    T.NestedField(1, "id_poligono", T.StringType(), required=True),
    T.NestedField(2, "data_penultima_ingestao", T.StringType(), required=False),
    T.NestedField(3, "data_ultima_ingestao", T.StringType(), required=True),
    identifier_field_ids=[1]
)

In [8]:
table = glue_catalog.create_table(
    identifier=(DATABASE, tb_name),
    schema=schema,
    location=table_location,
    sort_order=SortOrder(
        fields=[
            SortField(1, IdentityTransform(), ascending=True),
        ]
    ),
    properties={
        "format-version": "2",
        "write.format.default": "parquet",
        "write.parquet.compression-codec": "snappy",
        "write.parquet.page-size": "1048576",  # 1 MB
        "write.parquet.dictionary-enabled": "true",
        "write.parquet.dictionary-page-size": "1048576",  # 1 MB
        "write.parquet.block-size": "268435456",  # 256 MB
        "write.parquet.enable-encoding": "true",
        "write.parquet.enable-statistics": "true",
        "write.parquet.enable-bloom-filter": "true",
        "write.parquet.bloom-filter-encoding": "utf8",
    }
)

## Insert data

In [8]:
table = glue_catalog.load_table((DATABASE, tb_name))

In [9]:
arrow_schema = pa.schema(
    [
        pa.field("id_poligono", pa.string(), nullable=False),
        pa.field("data_penultima_ingestao", pa.string(), nullable=True),
        pa.field("data_ultima_ingestao", pa.string(), nullable=False),
    ]
)

In [ ]:
table_values = [
    {"id_poligono": "id_001", "data_penultima_ingestao": "2025-08-01 12:30:10", "data_ultima_ingestao": "2025-08-06 12:30:10"},
]

In [133]:
pa_table = pa.Table.from_pylist(
    table_values,
    schema=arrow_schema
)

In [134]:
pa_table.to_pandas().head()

,id_poligono,poligono,data_plantio,anomesdia
0,id_001,"POLYGON ((-46.556983701 -23.612093836, -46.556...",2023-06-20,20250810
1,id_002,"POLYGON ((-46.350654 -23.507674999, -46.350485...",2023-07-10,20250810
2,id_003,"POLYGON ((-46.915293 -23.513940999, -46.915253...",2023-08-01,20250810
3,id_004,"POLYGON ((-36.983988539 -10.682781162, -36.982...",2023-08-01,20250810
4,id_005,"POLYGON ((-43.402914028 -22.800304262, -43.401...",2023-08-01,20250810


In [135]:
table.append(pa_table)

/home/apolo/anaconda3/envs/itau/lib/python3.10/site-packages/pyiceberg/io/pyarrow.py:2532: UserWarning: Parquet writer option(s) ['write.parquet.bloom-filter-max-bytes'] not implemented
  warnings.warn(f"Parquet writer option(s) {unsupported_keys} not implemented")
